In [1]:
# impotr required library
import pandas as pd
import time
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [6]:
# loading chrome by automated
dr1=webdriver.Chrome(r'F:\Python\chromedriver.exe')

### Scarping reviews from flipkart

In [7]:
# load wensite
url='https://www.flipkart.com/'
dr1.get(url)

In [8]:
dr1.maximize_window()

In [9]:
# close login window
dr1.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [10]:
#create list of product
product=['phones','laptops','smart watches','moniters']
urls=[]

In [11]:
# Run for loop for fetching all produccts urls
for i in product:
    search=dr1.find_element_by_xpath('//div[@class="_3OO5Xc"]/input') # search products
    search.send_keys(Keys.CONTROL,'a')
    search.send_keys(Keys.BACKSPACE)
    search.send_keys(i)
    dr1.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    pages=[]
    for i in dr1.find_elements_by_xpath('//nav[@class="yFHi8N"]/a'): # getting urls of 4 pages
        pages.append(i.get_attribute('href'))
    for i in pages[0:4]:
        dr1.get(i)
        time.sleep(2)
        link=dr1.find_elements_by_xpath("//a[@class='_1fQZEK']") #get 4 pages product links 
        for i in link:
            urls.append(i.get_attribute('href'))
        time.sleep(2)    

In [12]:
len(urls)

384

In [13]:
rating=[]
review=[]
review_comments=[]

In [14]:
# scraping function for reviews,rating,comments
def scrap():    
        for i in dr1.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_comments.append(i.text)
        for i in dr1.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            review.append(i.text)
        for i in dr1.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            rating.append(i.text)
        return

In [15]:
# run for loop for fetching details from product urls
for i in urls:
    dr1.get(i)
    time.sleep(2)
    for i in range(2):
        dr1.execute_script("window.scrollBy(0,6000)")  # scroll down
        time.sleep(2)
        #clicking on all reviews
    try:
        btn=dr1.find_element_by_xpath("//div[@class='_2c2kV-']/following::a") # get review from front page
        lnk = btn.get_attribute('href')
        dr1.get(lnk)
        time.sleep(4)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=dr1.find_elements_by_xpath("//nav[@class='yFHi8N']/a") # get review from multiple pages
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:10]:
            dr1.get(i)
            time.sleep(5)
            scrap()
    except: continue

In [16]:
len(rating),len(review),len(review_comments)

(29873, 29873, 29873)

In [17]:
# make data frame
df1=pd.DataFrame({'review_title':review,'review_comments':review_comments,'rating':rating})

In [18]:
df1.head(15)

,review_title,review_comments,rating
0,Very Good,Thanks again Flipkart team the wonderful deliv...,4
1,Terrific,Very nice mobile and battry is very extra powa...,5
2,Terrific purchase,Camera 4/5 if u able to set it properly\nBatte...,5
3,Good choice,Positive sides :\n1. Battery is very very powe...,4
4,Fantastic,Excellent!!\nVery good mobile. I ordered it to...,5
5,Best in the market!,Best phone of this best valuable price..\nI th...,5
6,Fabulous!,Best phone superb design and colour and perfor...,5
7,Nice product,I bought it as a secondary device but was surp...,4
8,Good choice,It is a decent budgeted smart phone and main p...,4
9,Decent product,Display could have FHD....camera is ok.\nRest ...,3


In [81]:
# for amazon rating //div[@class='a-section review aok-relative']/div/div/div[2]/a[1]//span
# for review title //div[@class='a-section review aok-relative']/div/div/div[2]/a[2]//span
# for review comment //div[@class='a-section review aok-relative']/div/div/div[4]/span

### scrape data from amazon

In [40]:
# load webdriver
dr2=webdriver.Chrome(r'F:\Python\chromedriver.exe')
url2='https://www.amazon.in/'
dr2.get(url2)

In [41]:
# maximize window
dr2.maximize_window()

In [42]:
# product list
items=['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']
urls2=[]

In [49]:
for i in items:
    time.sleep(1)
    search=dr2.find_element_by_id('twotabsearchtextbox') # search items
    search.send_keys(Keys.CONTROL,'a')
    search.send_keys(Keys.BACKSPACE)
    search.send_keys(i)
    dr2.find_element_by_id("nav-search-submit-text").click()
    time.sleep(2)
    start=0
    end=4
    for page in range(start,end):                                #for loop for scrapping 5 pages
        time.sleep(3)
        link=dr2.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
        for i in link:
            urls2.append(i.get_attribute('href'))               # get links of products
        dr2.execute_script("window.scrollBy(0,1000)")
        time.sleep(5)                                                                                            # //li[@class='a-last']/./a
        nxt_button=dr2.find_element_by_xpath("//li[@class='a-last']/./a")     #scraping the list of buttons from the page
        nxt_button.click()
        time.sleep(3)

In [50]:
len(urls2),#urls2.clear()

(801,)

In [51]:
rating1=[]
review1=[]
review_comment1=[]

In [52]:
#sraping function
def scrap2():
    for i in dr2.find_elements_by_xpath("//div[@class='a-section review aok-relative']/div/div/div[2]/a[1]//span[1]"):
        rating1.append(i.get_attribute('innerHTML'))
    for i in dr2.find_elements_by_xpath("//div[@class='a-section review aok-relative']/div/div/div[4]/span[1]"):
        review_comment1.append(i.text)
    for i in dr2.find_elements_by_xpath("//div[@class='a-section review aok-relative']/div/div/div[2]/a[2]//span[1]"):
        review1.append(i.text)
    return

In [59]:
for i in urls2:
    dr2.get(i)
    time.sleep(2)
    dr2.execute_script('window.scrollBy(0,7000)')
    time.sleep(2)
    try:
        btn=dr2.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']")
        btn.click()
        time.sleep(3)
        scrap2()
        time.sleep(3)
    except NoSuchElementException:
        pass
    dr2.execute_script('window.scrollBy(0,6000)')
    time.sleep(4)

In [60]:
len(rating1),len(review1),len(review_comment1)

(5668, 5668, 5763)

In [58]:
#rating1.clear(),review1.clear(),review_comment1.clear()

(None, None, None)

In [61]:
r1=review_comment1[0:5668]
len(r1)

5668

In [63]:
# make data frame
df2=pd.DataFrame({'review_title':review1,'review_comments':r1,'rating':rating1,})

In [64]:
df2

,review_title,review_comments,rating
0,Go For it....!!,- I am using this laptop for about 15 days and...,5.0 out of 5 stars
1,Faulty,I got a faulty product the charger wasn’t work...,1.0 out of 5 stars
2,Best For The Price,one of the best thermals out there intels new ...,5.0 out of 5 stars
3,Amazing performance,Good in everything,5.0 out of 5 stars
4,Best RTX Laptop under 70k,This is the best rtx 3050 lap u can get under ...,5.0 out of 5 stars
...,...,...,...
5663,More Range - More connected devices,This router is literally a striped down versio...,5.0 out of 5 stars
5664,Faulty Product,Product is ok. Brought the costlier one thinki...,1.0 out of 5 stars
5665,Does what a basic router should do but for hig...,This Review is in depth after a full year of u...,5.0 out of 5 stars
5666,Mostly router decreases speed from 50Mbps to 5...,"I purchased this for my office use. Hence, you...",1.0 out of 5 stars


In [65]:
# merge two data frame
df=pd.concat([df1,df2],ignore_index=False)

In [66]:
df

,review_title,review_comments,rating
0,Very Good,Thanks again Flipkart team the wonderful deliv...,4
1,Terrific,Very nice mobile and battry is very extra powa...,5
2,Terrific purchase,Camera 4/5 if u able to set it properly\nBatte...,5
3,Good choice,Positive sides :\n1. Battery is very very powe...,4
4,Fantastic,Excellent!!\nVery good mobile. I ordered it to...,5
...,...,...,...
5663,More Range - More connected devices,This router is literally a striped down versio...,5.0 out of 5 stars
5664,Faulty Product,Product is ok. Brought the costlier one thinki...,1.0 out of 5 stars
5665,Does what a basic router should do but for hig...,This Review is in depth after a full year of u...,5.0 out of 5 stars
5666,Mostly router decreases speed from 50Mbps to 5...,"I purchased this for my office use. Hence, you...",1.0 out of 5 stars


In [67]:
# save csv format
df.to_csv('review_comments.csv')

In [68]:
# save to excel format
df.to_excel('review_comments.xlsx')